<a href="" rel="Topology"><img src="../data/demo-test/topology.png" alt="" /></a>

#### 1) We have pre-collected the training data. Now we are going to select one victim and corrupt (cache or link)

In [2]:
#victim = "syr-cache"
#victim = "uc-router-n0_Link12"
victim = "uc-compute-c1_Link4"


cmd = "set_victim.sh {}".format(victim)

import subprocess
ssh_control = "ssh -i ~/.ssh/iris_rsa root@129.7.98.66 /root/IRIS/orchestra/"
args = "{}{}".format(ssh_control, cmd).split()
print(subprocess.check_output(args, encoding='UTF-8', stderr=subprocess.STDOUT))
victim = victim.replace('-','.') # to sync with what data analysis will use
victim = victim.replace('_','.')

victim is set: 
export uc-compute-c1_Link4=192.168.17.3



#### 2) Starting the workflows from 4 submit sites (Beware following sell takes 12 mins), and collect data from pagasus elastic search

In [ ]:
cmd = "wf_test_v2.sh"

args = "{}{}".format(ssh_control, cmd).split()
print(subprocess.check_output(args, encoding='UTF-8', stderr=subprocess.STDOUT))

#### 3) let's download the file as test data.

In [3]:
filename = "01v2_02_20210909_1745PM" # replace this

remote_file = "/var/iris_results/{}/{}_full.csv".format(filename,filename)
test_file = "../data/demo-test/test.csv"

# copy the full csv and zip to local
cmd = "scp -i ~/.ssh/iris_rsa root@129.7.98.66:{} {}".format(remote_file, test_file)
subprocess.check_output( cmd.split(), encoding='UTF-8', stderr=subprocess.STDOUT)
cmd = "scp -i ~/.ssh/iris_rsa root@129.7.98.66:/var/iris_results/{}.tar.gz ../data/demo-test/".format(filename)
subprocess.check_output( cmd.split(), encoding='UTF-8', stderr=subprocess.STDOUT)

''

#### 4) Read the pre-collected train data and import some functions before applying to the model.

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [5]:
def file_process(data_file, info=False):
    input_file = data_file
    df = pd.read_csv(input_file, header = 0)
    #df.dropna(inplace = True) 
    df.fillna(1.0/500.0) #the corruption rate for the cache failures is all set to be "NaN"
    df['corrupt_label']=df['corrupt_label'].str.replace('-','.')
    df['flow'] = df['submit_host']+'-'+df['execution_host']
    df['FM']=df['transfer_success']+df['checksum_success']
    
    df_failure = df[df.checksum_success==False]
    df_dummy = pd.get_dummies(df_failure[['submit_host', 'execution_host','flow','transfer_success', 'checksum_success','src_label','dst_label','bytes','corrupt_rate','corrupt_label']], prefix=['submit_host', 'execution_host','src_label','dst_label','flow'], columns=['submit_host', 'execution_host','src_label','dst_label','flow'])
    #df_dummy['bytes']=df_dummy['bytes'].str[0:-4]
    df_dummy['bytes']=pd.to_numeric(df_dummy['bytes'], errors='coerce')
    df_dummy['corrupt_rate']=pd.to_numeric(df_dummy['corrupt_rate'], errors='coerce')
    df_dummy = df_dummy.replace(np.nan, 0, regex=True)
    y_complete=df_dummy['corrupt_label']
    X_complete=df_dummy.drop(['corrupt_label'],axis=1)
    
    if info:
        original_headers = list(df.columns.values)
        print("Original shape:")
        print(original_headers)
        print("Original shape:"+str(df.shape))
        dummy_headers = list(df_dummy.columns.values)
        print("Encoded shape:")
        print(dummy_headers)
        print(str(df_dummy.shape))
    
    return df,df_dummy,X_complete,y_complete
    
    #return df,df_dummy

In [6]:
#input_file = "../data/exogeni/var/iris_results/01v2_02_20210115_0445AM/01v2_02_20210115_0445AM_full.csv"
input_file = "../data/exogeni/var2/iris_results/01v2_02_20210125_1336PM/01v2_02_20210125_1336PM_full.csv"
df_ori,df_dummy,X_complete, y_complete=file_process(input_file,True)
#df_ori,df_dummy=file_process(input_file,False)
y_complete=df_dummy['corrupt_label']
X_complete=df_dummy.drop(['corrupt_label'],axis=1)
df_training=[df_dummy]

training_dataset={"complete":{"X":X_complete,
                 "y":y_complete}}

Original shape:
['root_xwf_id', 'job_id', 'start_time', 'end_time', 'submit_host', 'submit_user', 'execution_host', 'execution_user', 'job_type', 'job_exit_code', 'bytes', 'lfn', 'src_label', 'src_url', 'src_proto_host', 'dst_label', 'dst_url', 'dst_proto_host', 'transfer_success', 'checksum_success', 'actual_checksum', 'expected_checksum', 'scenario', 'corrupt_label', 'corrupt_start', 'corrupt_end', 'corrupt_rate', 'flow', 'FM']
Original shape:(45291, 29)
Encoded shape:
['transfer_success', 'checksum_success', 'bytes', 'corrupt_rate', 'corrupt_label', 'submit_host_syr-submit', 'submit_host_uc-submit', 'submit_host_ucsd-submit', 'submit_host_unl-submit', 'execution_host_syr-compute-c0', 'execution_host_syr-compute-c1', 'execution_host_uc-compute-c0', 'execution_host_uc-compute-c1', 'execution_host_ucsd-compute-c0', 'execution_host_ucsd-compute-c1', 'execution_host_unl-compute-c0', 'execution_host_unl-compute-c1', 'src_label_syr', 'src_label_uc', 'src_label_ucsd', 'src_label_unl', 'dst_

/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


In [7]:
#test_file = "../data/exogeni/var2/iris_results/01v2_02_20210125_1336PM/01v2_02_20210125_1336PM_full.csv"
#test_file = "../data/demo-test/01v2_02_20210831_1741PM_full_syr-cache.csv"
#test_file = "../data/demo-test/01v2_02_20210901_1241PM_full_uc-router-n0-link.csv"
test_file = "../data/demo-test/test.csv"
df_t_ori,df_t_dummy,X_test, y_test=file_process(test_file,True)
#df_t_ori,df_t_dummy=file_process(test_file,True)
#y_test=df_t_dummy['corrupt_label']
#X_test=df_t_dummy.drop(['corrupt_label'],axis=1)

Original shape:
['root_xwf_id', 'job_id', 'start_time', 'end_time', 'submit_host', 'submit_user', 'execution_host', 'execution_user', 'job_type', 'job_exit_code', 'bytes', 'lfn', 'src_label', 'src_url', 'src_proto_host', 'dst_label', 'dst_url', 'dst_proto_host', 'transfer_success', 'checksum_success', 'actual_checksum', 'expected_checksum', 'scenario', 'corrupt_label', 'corrupt_start', 'corrupt_end', 'corrupt_rate', 'flow', 'FM']
Original shape:(632, 29)
Encoded shape:
['transfer_success', 'checksum_success', 'bytes', 'corrupt_rate', 'corrupt_label', 'submit_host_syr-submit', 'submit_host_uc-submit', 'submit_host_ucsd-submit', 'submit_host_unl-submit', 'execution_host_uc-compute-c1', 'src_label_syr', 'src_label_uc', 'src_label_ucsd', 'src_label_unl', 'dst_label_uc', 'flow_syr-submit-uc-compute-c1', 'flow_uc-submit-uc-compute-c1', 'flow_ucsd-submit-uc-compute-c1', 'flow_unl-submit-uc-compute-c1']
(58, 19)


/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


In [8]:
def missing_feature(X_complete, X_test):
    print("X_complete:"+str(len(X_complete.columns)))
    print("X_test:" + str(len(X_test.columns)))
    c_t_missing=[]
    for c_c in X_complete.columns:
        if c_c not in X_test.columns:
            #print(c_c+" c_c not in test!")
            c_t_missing.append(c_c)
            
    c_c_missing=[]
    for c_t in X_test.columns:
        if c_t not in X_complete.columns:
            #print(c_t+" c_t not in complete!")
            c_c_missing.append(c_t)

    for c_c in c_t_missing:
        #print(c_t+" c_t dropping!")
        X_test[c_c]='0'
        
    for c_t in c_c_missing:
        #print(c_t+" c_t dropping!")
        X_test=X_test.drop(columns=c_t)
        
    X_complete, X_test = X_complete.align(X_test, join='inner', axis=1)
    
    print("After imputation:"+str(len(X_test.columns)))
    
    return X_test

In [9]:
X_test = missing_feature(X_complete, X_test)
df_t_dummy = missing_feature(df_dummy, df_t_dummy)

#df_dummy,df_t_dummy = df_dummy.align(df_t_dummy, join='inner', axis=1)
#X_complete, X_test = X_complete.align(X_test, join='inner', axis=1)

df_t=[df_t_dummy]
testing_dataset={"testing":{"X":X_test,
                 "y":y_test}}

X_complete:55
X_test:18
After imputation:55
X_complete:56
X_test:19
After imputation:56


In [10]:
def accuracy(model, test_data, classes, class_label, k):
    if(k==0): return null,null
    num = classes.size
    correct_class={}
    correct=0
    for label in classes:
        label_pred=[]
        isCorrect=False
        test_data_1=test_data[test_data[class_label]==label]
        if test_data_1.shape[0]==0:
            print("No match for {}.".format(label))
            correct=correct+1
            correct_class[label]=1
            continue
        else:
            print("label {}.".format(label))
        x_test_1=test_data_1.drop([class_label],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        #label_pred[0] = classes[label_index]
        for j in range(0,k): 
            #print(f'j={j}, label={label_index}')
            if(j==0):
                label_pred.append(classes[label_index])
            else:
                label_pred.append(classes[label_index_sort[(-1)*j-1]])
            #print(j,label_pred[j],label)
            if(label==label_pred[j]):
                isCorrect=True
                break
               
        #print(label_pred)
        #print(isCorrect)
        if isCorrect:
            print("predicted label:"+label)
            correct=correct+1
            correct_class[label]=1
        else:
            correct_class[label]=0
            print("wrong label:"+label)
            for j in range(0,k): 
                print("predicted label "+str(j)+":"+label_pred[j])
    return correct_class, correct, correct/num 

In [11]:
def predict(model, test_data, classes, class_label):

    num = classes.size
    correct_class={}
    correct=0

    print("classes are {}".format(classes))
    #what_we_corrupted = y_test.array[0]
    what_we_corrupted = "{} ".format(victim)
    print(what_we_corrupted)
    if what_we_corrupted not in classes.tolist():
        what_we_corrupted=victim.replace('-','.').split('.')[0]
    print(what_we_corrupted)
    pos = classes.tolist().index(what_we_corrupted)
    print("what_we_corrupted: {}, which is at index {}".format(what_we_corrupted, pos))
    
    ourlabel = [what_we_corrupted]
    print("ourlabel = {}".format(ourlabel))
    for label in ourlabel:
        print("class_label/label = {}/{}".format(class_label,label))
        label_pred=[]
        isCorrect=False
        
        test_data_1=test_data[test_data[class_label]==label]
        if test_data_1.shape[0]==0:
            print("No match {}".format(label))
            correct=correct+1
            correct_class[label]=1
            continue
        x_test_1=test_data_1.drop([class_label],axis=1)
        test_prob_1 = model.predict_proba(x_test_1)
        label_array=test_prob_1.mean(axis=0)
        print("label_array = {}".format(label_array))
        #label_array=np.nanmean(np.where(test_rf_prob_1!=0,test_rf_prob_1,np.nan),0)
        label_index = label_array.argmax()
        label_index_sort = label_array.argsort()
        rank = classes.size - label_index_sort.tolist().index(pos)
        top_k = 3
        if rank > 3:
            top_k = rank
        print("top_k = {}".format(top_k))
        for x in range (1, top_k + 1):
            guess = classes[label_index_sort[(-1)*x]]
            print("{}th prediction: {}".format(x, guess))
        print('\n"{}" is rank {} in prediction'.format(what_we_corrupted, rank))


In [12]:
def train_dt(classifier, df, training_dataset,class_label):
    i=0
    for k, d in training_dataset.items():
        X=d["X"]
        y=d['y']
        clf = classifier
        clf_model=clf.fit(X,y)
        pred=clf.predict(X)
        balanced_accu=balanced_accuracy_score(y, pred)
        f1=f1_score(y, pred,average='weighted')
        print(str(k)+ "label_size:" + str(clf.classes_.size) + ":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
        for j in range(1,4):
            c,correct,accu=accuracy(clf_model, df[i], clf.classes_, class_label, j)
            print("Top-"+str(j)+" Accu="+str(accu)) 
    return clf_model

In [13]:
def test_dt(clf_model,df_t, testing_dataset,class_label):
    i=0
    for k, d in testing_dataset.items():
        y_t = d['y']
        X_t = d['X']
        pred=clf_model.predict(X_t)
        balanced_accu=balanced_accuracy_score(y_t, pred)
        #f1=f1_score(y_t, pred,average='weighted')
        #print(":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
        for j in range(1,4):
            c,correct,accu=accuracy(clf_model, df_t[i], clf_model.classes_,class_label,j)
            print("Top-"+str(j)+" Accu="+str(accu))
        predict(clf_model, df_t[i], clf_model.classes_,class_label)
        i=i+1

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
#import graphviz 
from sklearn.calibration import CalibratedClassifierCV

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import multilabel_confusion_matrix,balanced_accuracy_score,f1_score

# 1. Try the top-k classification with the corrupted flows

In [ ]:
clf_model = train_dt(RandomForestClassifier(max_depth=20, random_state=0),df_training,training_dataset, 'corrupt_label')

In [16]:
test_dt(clf_model,df_t,testing_dataset, 'corrupt_label')

No match for cenic.Link12 .
No match for cenic.Link9 .
No match for esnet.Link1 .
No match for esnet.Link2 .
No match for esnet.Link22 .
No match for esnet.Link7 .
No match for internet2.Link14 .
No match for internet2.Link2 .
No match for starlight.Link24 .
No match for starlight.Link6 .
No match for starlight.Link7 .
No match for starlight.Link9 .
No match for syr.cache .
No match for syr.compute.c0.Link26 .
No match for syr.router.n2.Link24 .
No match for uc.cache .
No match for uc.compute.c0.Link3 .
label uc.compute.c1.Link4 .
predicted label:uc.compute.c1.Link4 
No match for uc.router.n0.Link12 .
No match for ucsd.cache .
No match for ucsd.compute.c0.Link23 .
No match for ucsd.compute.c1.Link13 .
No match for ucsd.router.n3.Link22 .
No match for unl.cache .
No match for unl.compute.c1.Link16 .
No match for unl.router.n1.Link14 .
Top-1 Accu=1.0
No match for cenic.Link12 .
No match for cenic.Link9 .
No match for esnet.Link1 .
No match for esnet.Link2 .
No match for esnet.Link22 .
No

/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


# 2. Let's balance the data via oversampling

In [17]:
from imblearn.over_sampling import RandomOverSampler,SMOTE, ADASYN,SMOTENC
from imblearn.combine import SMOTEENN, SMOTETomek

def train_over_sampling(classifier,df,training_dataset,class_label,test=False):
    i=0
    over_sampling = [RandomOverSampler(random_state=0),
        #SMOTE(random_state=0),
        #SMOTEENN(random_state=0),
        #SMOTETomek(random_state=0),
        #SMOTENC(categorical_features=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43], random_state=0)

    ]
    for k, d in training_dataset.items():
        for s in range(0,len(over_sampling)):
            X=d["X"]
            y=d['y']
            print("sampling:"+str(s)+":"+str(over_sampling[s]))
            os = over_sampling[s]
            X_resampled, y_resampled = os.fit_resample(X, y)
            #clf = RandomForestClassifier(max_depth=20, random_state=0)
            clf = classifier
            clf_model=clf.fit(X_resampled, y_resampled)
            
            pred=clf.predict(X)
            balanced_accu=balanced_accuracy_score(y, pred)
            f1=f1_score(y, pred,average='weighted')
            print(str(k)+":balanced_accu="+str(balanced_accu)+":F1-Score="+str(f1))
            
            for j in range(1,4):
                if test:
                    c,correct,accu=accuracy(clf_model, df_t[i], clf.classes_,class_label,j)
                else:
                    c,correct,accu=accuracy(clf_model, df[i], clf.classes_,class_label,j)
                print("Top-"+str(j)+" Accu="+str(accu)) 
        i=i+1
        
    return clf_model

In [18]:
#df_ori,df_dummy=file_process(input_file,True)
#y_complete=df_dummy['corrupt_label']
#X_complete=df_dummy.drop(['corrupt_label'],axis=1)

df_ori,df_dummy,X_complete, y_complete=file_process(input_file,True)
df=[df_dummy]

training_dataset={"complete":{"X":X_complete,
                 "y":y_complete}}

clf_model = train_over_sampling(RandomForestClassifier(max_depth=20, random_state=0),df,training_dataset, 'corrupt_label')

/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


Original shape:
['root_xwf_id', 'job_id', 'start_time', 'end_time', 'submit_host', 'submit_user', 'execution_host', 'execution_user', 'job_type', 'job_exit_code', 'bytes', 'lfn', 'src_label', 'src_url', 'src_proto_host', 'dst_label', 'dst_url', 'dst_proto_host', 'transfer_success', 'checksum_success', 'actual_checksum', 'expected_checksum', 'scenario', 'corrupt_label', 'corrupt_start', 'corrupt_end', 'corrupt_rate', 'flow', 'FM']
Original shape:(45291, 29)
Encoded shape:
['transfer_success', 'checksum_success', 'bytes', 'corrupt_rate', 'corrupt_label', 'submit_host_syr-submit', 'submit_host_uc-submit', 'submit_host_ucsd-submit', 'submit_host_unl-submit', 'execution_host_syr-compute-c0', 'execution_host_syr-compute-c1', 'execution_host_uc-compute-c0', 'execution_host_uc-compute-c1', 'execution_host_ucsd-compute-c0', 'execution_host_ucsd-compute-c1', 'execution_host_unl-compute-c0', 'execution_host_unl-compute-c1', 'src_label_syr', 'src_label_uc', 'src_label_ucsd', 'src_label_unl', 'dst_

In [19]:
test_dt(clf_model,df_t,testing_dataset, 'corrupt_label')

No match for cenic.Link12 .
No match for cenic.Link9 .
No match for esnet.Link1 .
No match for esnet.Link2 .
No match for esnet.Link22 .
No match for esnet.Link7 .
No match for internet2.Link14 .
No match for internet2.Link2 .
No match for starlight.Link24 .
No match for starlight.Link6 .
No match for starlight.Link7 .
No match for starlight.Link9 .
No match for syr.cache .
No match for syr.compute.c0.Link26 .
No match for syr.router.n2.Link24 .
No match for uc.cache .
No match for uc.compute.c0.Link3 .
label uc.compute.c1.Link4 .
predicted label:uc.compute.c1.Link4 
No match for uc.router.n0.Link12 .
No match for ucsd.cache .
No match for ucsd.compute.c0.Link23 .
No match for ucsd.compute.c1.Link13 .
No match for ucsd.router.n3.Link22 .
No match for unl.cache .
No match for unl.compute.c1.Link16 .
No match for unl.router.n1.Link14 .
Top-1 Accu=1.0
No match for cenic.Link12 .
No match for cenic.Link9 .
No match for esnet.Link1 .
No match for esnet.Link2 .
No match for esnet.Link22 .
No

/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


# 3. Try the coarser per-site classification

In [20]:
#count the number of labels in the data and the number of labels that actually caused errors.
def num_label(df, df_dummy, site):
    site_ori = df[site]
    counter_site = Counter(site_ori)
    print("original num_label:" + str(len(counter_site)))
    #for k,v in counter_site.items():
    #    print(k)
    
    site_dummy = df_dummy[site]
    counter_site_dummy = Counter(site_dummy)
    print("recorded num_label:" + str(len(counter_site_dummy)))

    site_name = df[site].str.split('.', n=1, expand = True)
    site_dummy_name = df[site].str.split('.', n=1, expand = True)
    
    return counter_site, counter_site_dummy,site_name,site_dummy_name

In [21]:
counter_site,counter_site_dummy,site_name,site_dummy_name=num_label(df_ori,df_dummy,'corrupt_label')
df_ori['site'] = site_name[0]
df_dummy['site'] = site_dummy_name[0]
df_dummy=df_dummy.drop(['corrupt_label'],axis=1)
y_complete=df_dummy['site']
X_complete=df_dummy.drop(['site'],axis=1)

df=[df_dummy]
    
training_dataset={"complete":{"X":X_complete,
                 "y":y_complete}}

site_model = train_dt(RandomForestClassifier(max_depth=20, random_state=0),df,training_dataset, 'site')

original num_label:49
recorded num_label:26
completelabel_size:8:balanced_accu=0.6093384531968332:F1-Score=0.6106753029121078
label cenic.
predicted label:cenic
label esnet.
predicted label:esnet
label internet2.
predicted label:internet2
label starlight.
predicted label:starlight
label syr.
predicted label:syr
label uc.
predicted label:uc
label ucsd.
predicted label:ucsd
label unl.
predicted label:unl
Top-1 Accu=1.0
label cenic.
predicted label:cenic
label esnet.
predicted label:esnet
label internet2.
predicted label:internet2
label starlight.
predicted label:starlight
label syr.
predicted label:syr
label uc.
predicted label:uc
label ucsd.
predicted label:ucsd
label unl.
predicted label:unl
Top-2 Accu=1.0
label cenic.
predicted label:cenic
label esnet.
predicted label:esnet
label internet2.
predicted label:internet2
label starlight.
predicted label:starlight
label syr.
predicted label:syr
label uc.
predicted label:uc
label ucsd.
predicted label:ucsd
label unl.
predicted label:unl
Top-

In [22]:
counter_site_t,counter_site_t_dummy,site_t_name,site_t_dummy_name=num_label(df_t_ori,df_t_dummy,'corrupt_label')
df_t_ori['site'] = site_t_name[0]
df_t_dummy['site'] = site_t_dummy_name[0]
if 'corrupt_label' in df_t_dummy:
    df_t_dummy=df_t_dummy.drop(['corrupt_label'],axis=1)
y_t=df_t_dummy['site']
X_t=df_t_dummy.drop(['site'],axis=1)

df_t=[df_t_dummy]
    
testing_dataset={"complete":{"X":X_t,
                 "y":y_t}}
test_dt(site_model,df_t,testing_dataset, 'site')

original num_label:1
recorded num_label:1
No match for cenic.
No match for esnet.
No match for internet2.
No match for starlight.
No match for syr.
label uc.
predicted label:uc
No match for ucsd.
No match for unl.
Top-1 Accu=1.0
No match for cenic.
No match for esnet.
No match for internet2.
No match for starlight.
No match for syr.
label uc.
predicted label:uc
No match for ucsd.
No match for unl.
Top-2 Accu=1.0
No match for cenic.
No match for esnet.
No match for internet2.
No match for starlight.
No match for syr.
label uc.
predicted label:uc
No match for ucsd.
No match for unl.
Top-3 Accu=1.0
classes are ['cenic' 'esnet' 'internet2' 'starlight' 'syr' 'uc' 'ucsd' 'unl']
uc.compute.c1.Link4 
uc
what_we_corrupted: uc, which is at index 5
ourlabel = ['uc']
class_label/label = site/uc
label_array = [0.         0.18086551 0.06026965 0.06896552 0.         0.68989933
 0.         0.        ]
top_k = 3
1th prediction: uc
2th prediction: esnet
3th prediction: starlight

"uc" is rank 1 in predi

/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


# 4. try the probability based model

In [23]:
def prob_dict(df_c,group):
    target_flow=df_c[group]
    counter_flow = Counter(target_flow)
    flow_prob_dict={}
    for i,j in counter_flow.items():
    #print(i+":"+str(j)+":"+str(len(target_flow)))
        per_flow = j / len(target_flow) * 100
        #print('flow=%s, Count=%d, Per__flow=%.3f%%' % (i, j, per_flow))
        df_flow=df_c[(df_c[group]==i)]
        flow_count=df_flow['checksum_success'].value_counts(normalize=True)
        #print(flow_count.index.tolist())
        #print(flow_count.values.tolist())
        flow_count_dict=flow_count.to_dict()
        #print(flow_count_dict)
        i_group=group+"_"+i
        if 0 in flow_count_dict:
            flow_prob_dict[i_group]=flow_count_dict[0]
        else:
            flow_prob_dict[i_group]=0
        #print("{}:{}".format(i,flow_prob_dict[i]))
    return flow_prob_dict

In [24]:
def generate_prob_dict(df, target_name,groups):
    # summarize the class distribution
    #target = df.values[:,-5]
    target = df[target_name]
    counter = Counter(target)
    label_prob_dict={}
    for k,v in counter.items():
        per = v / len(target) * 100
        #print('Class=%s, Count=%d, Percentage=%.3f%%' % (k, v, per))
        df_c=df[(df[target_name]==k)]
        flow_prob_dict={}
        for group in groups:
            prob=prob_dict(df_c,group)
            flow_prob_dict={**flow_prob_dict,**prob}
        
        label_prob_dict[k]=flow_prob_dict
    return label_prob_dict

In [25]:
df_ori,df_dummy,X_complete, y_complete=file_process(input_file,True)
groups=["flow","src_label","dst_label"]
label_prob_dict=generate_prob_dict(df_ori,"corrupt_label",groups)
df_prob = pd.DataFrame.from_dict(label_prob_dict, orient='index')
df_prob = df_prob.fillna(0)
df_prob = df_prob[(df_prob.T !=0).any()] #drop rows with all zeros: no corruption labels
df_prob=df_prob.sort_index() #sort the index
X=df_prob.to_numpy()
X=np.nan_to_num(X)
#print(X)
y=df_prob.index
#print(y.shape)

/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


Original shape:
['root_xwf_id', 'job_id', 'start_time', 'end_time', 'submit_host', 'submit_user', 'execution_host', 'execution_user', 'job_type', 'job_exit_code', 'bytes', 'lfn', 'src_label', 'src_url', 'src_proto_host', 'dst_label', 'dst_url', 'dst_proto_host', 'transfer_success', 'checksum_success', 'actual_checksum', 'expected_checksum', 'scenario', 'corrupt_label', 'corrupt_start', 'corrupt_end', 'corrupt_rate', 'flow', 'FM']
Original shape:(45291, 29)
Encoded shape:
['transfer_success', 'checksum_success', 'bytes', 'corrupt_rate', 'corrupt_label', 'submit_host_syr-submit', 'submit_host_uc-submit', 'submit_host_ucsd-submit', 'submit_host_unl-submit', 'execution_host_syr-compute-c0', 'execution_host_syr-compute-c1', 'execution_host_uc-compute-c0', 'execution_host_uc-compute-c1', 'execution_host_ucsd-compute-c0', 'execution_host_ucsd-compute-c1', 'execution_host_unl-compute-c0', 'execution_host_unl-compute-c1', 'src_label_syr', 'src_label_uc', 'src_label_ucsd', 'src_label_unl', 'dst_

In [26]:
clf_prob = RandomForestClassifier(max_depth=20, random_state=0)
clf_prob.fit(X,y)
clf_prob.score(X,y)

1.0

In [28]:
df_t_ori,df_t_dummy,X_t_complete, y_t_complete=file_process(test_file,True)
df_t_ori = missing_feature(df_ori, df_t_ori)
df_t_dummy = missing_feature(df_dummy, df_t_dummy)
groups=["flow","src_label","dst_label"]
label_prob_dict_test=generate_prob_dict(df_t_ori,"corrupt_label",groups)
df_prob_test = pd.DataFrame.from_dict(label_prob_dict_test, orient='index')

df_prob_test = missing_feature(df_prob, df_prob_test)

df_prob_test = df_prob_test.fillna(0)
df_prob_test = df_prob_test[(df_prob_test.T !=0).any()] #drop rows with all zeros: no corruption labels
#df_prob,df_prob_test = df_prob.align(df_prob_test, join='inner', axis=1)
df_prob_test = df_prob_test.sort_index() #sort the table index

X_t=df_prob_test.to_numpy()
print(X_t.shape)
X_t=np.nan_to_num(X_t)
print(X_t.shape)
y_t=df_prob_test.index
#print(y.shape)
clf_prob.score(X_t,y_t)

Original shape:
['root_xwf_id', 'job_id', 'start_time', 'end_time', 'submit_host', 'submit_user', 'execution_host', 'execution_user', 'job_type', 'job_exit_code', 'bytes', 'lfn', 'src_label', 'src_url', 'src_proto_host', 'dst_label', 'dst_url', 'dst_proto_host', 'transfer_success', 'checksum_success', 'actual_checksum', 'expected_checksum', 'scenario', 'corrupt_label', 'corrupt_start', 'corrupt_end', 'corrupt_rate', 'flow', 'FM']
Original shape:(632, 29)
Encoded shape:
['transfer_success', 'checksum_success', 'bytes', 'corrupt_rate', 'corrupt_label', 'submit_host_syr-submit', 'submit_host_uc-submit', 'submit_host_ucsd-submit', 'submit_host_unl-submit', 'execution_host_uc-compute-c1', 'src_label_syr', 'src_label_uc', 'src_label_ucsd', 'src_label_unl', 'dst_label_uc', 'flow_syr-submit-uc-compute-c1', 'flow_uc-submit-uc-compute-c1', 'flow_ucsd-submit-uc-compute-c1', 'flow_unl-submit-uc-compute-c1']
(58, 19)
X_complete:29
X_test:29
After imputation:29
X_complete:56
X_test:19
After imputati

/Users/yxin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


1.0

In [29]:
clf_prob.predict([X_t[0]])

array(['uc.compute.c1.Link4 '], dtype=object)

In [30]:
y_t[0]

'uc.compute.c1.Link4 '